# В данной работе будет исследоваться:
1. Зависимость количества ДТП от ям\неровностей на дороге
2. ...


## Installing dependecies, importing some of them and setting global variables

In [134]:
%pip install -qU pip setuptools pandas numpy plotly rich tqdm geopandas geojson bs4 requests ipywidgets==7.7.1 jupyterlab_widgets pydantic jellyfish shapely chardet
!git clone https://github.com/simp37/Russia_geoJSON

Found existing installation: numba 0.56.4
Uninstalling numba-0.56.4:
  Successfully uninstalled numba-0.56.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
fatal: destination path 'Russia_geoJSON' already exists and is not an empty directory.


In [3]:
import pickle
from rich import print
from tqdm.notebook import tqdm
from pathlib import Path
# from google.colab import output
# output.enable_custom_widget_manager()

data_accidents_source_url = 'https://dtp-stat.ru/opendata'

## Downloading data
If you are already have data in `./data` folder - skip this section

### Download HTML table and save links to list
source link was added in import section:
```python
data_accidents_source_usr = 'https://dtp-stat.ru/opendata'
```

In [5]:
from bs4 import BeautifulSoup
import urllib.request

html_page = urllib.request.urlopen(data_accidents_source_url)
soup = BeautifulSoup(html_page, "html.parser")
download_links = []
for link in soup.findAll('a'):
    link_href = link.get('href')
    download_links.append(link_href) if '.geojson' in link_href else None
print(download_links)

[
    'https://cms.dtp-stat.ru/media/opendata/altaiskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/amurskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/arkhangelskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/astrakhanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/belgorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/brianskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/vladimirskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/volgogradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/vologodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/voronezhskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/evreiskaia-avtonomnaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/zabaikalskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/ivanovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/irkutskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kabardino-balkarskaia-respublika.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kaliningradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kaluzhskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kamchatskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/karachaevo-cherkesskaia-respublika.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kemerovskaia-oblast-kuzbass.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kirovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kostromskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/krasnodarskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/krasnoiarskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kurganskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/kurskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/leningradskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/lipetskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/magadanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/moskva.geojson',
    'https://cms.dtp-stat.ru/media/opendata/moskovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/murmanskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/nenetskii-avtonomnyi-okrug.geojson',
    'https://cms.dtp-stat.ru/media/opendata/nizhegorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/novgorodskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/novosibirskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/omskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/orenburgskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/orlovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/penzenskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/permskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/primorskii-krai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/pskovskaia-oblast.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-adygeia-adygeia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-altai.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-bashkortostan.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-buriatiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-dagestan.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-ingushetiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-kalmykiia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-kareliia.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-komi.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-krym.geojson',
    'https://cms.dtp-stat.ru/media/opendata/respublika-marii

In [33]:
regions = [link.replace('https://cms.dtp-stat.ru/media/opendata/', '').replace('.geojson', '') for link in download_links]
#print(regions)


russia_regions = [str(region.name).replace('.geojson', '') for region in Path('Russia_geoJSON/').glob('*.geojson')]

import difflib
import jellyfish
eq_regions = {}
for region in regions:
    _region = region.replace('-oblast', '').replace('respublika-', '')
    if _region in ['krym', 'sevastopol']:
        continue
    score = 100
    for russia_region in russia_regions:
        if jellyfish.levenshtein_distance(_region, russia_region.lower()) < score:
            score = jellyfish.levenshtein_distance(_region, russia_region.lower())
            eq_regions[region] = russia_region
print(eq_regions)


{
    'altaiskii-krai': 'Altayskiy_kray',
    'amurskaia-oblast': 'Amurskaya',
    'arkhangelskaia-oblast': 'Arkhangel_skaya',
    'astrakhanskaia-oblast': 'Astrakhanskaya',
    'belgorodskaia-oblast': 'Belgorodskaya',
    'brianskaia-oblast': 'Bryanskaya',
    'vladimirskaia-oblast': 'Vladimirskaya',
    'volgogradskaia-oblast': 'Volgogradskaya',
    'vologodskaia-oblast': 'Vologodskaya',
    'voronezhskaia-oblast': 'Voronezhskaya',
    'evreiskaia-avtonomnaia-oblast': 'Saratovskaya',
    'zabaikalskii-krai': 'Zabaykal-skiy_kray',
    'ivanovskaia-oblast': 'Ivanovskaya',
    'irkutskaia-oblast': 'Irkutskaya',
    'kabardino-balkarskaia-respublika': 'Kabardino-Balkarskaya',
    'kaliningradskaia-oblast': 'Kaliningradskaya',
    'kaluzhskaia-oblast': 'Kaluzhskaya',
    'kamchatskii-krai': 'kamchatskaya',
    'karachaevo-cherkesskaia-respublika': 'Karachayevo-Cherkesskaya',
    'kemerovskaia-oblast-kuzbass': 'kermerovskaya',
    'kirovskaia-oblast': 'Kirovskaya',
    'kostromskaia-oblast': 'Kostromskaya',
    'krasnodarskii-krai': 'Krasnodarskiy-kray',
    'krasnoiarskii-krai': 'Krasnodarskiy-kray',
    'kurganskaia-oblast': 'Kurganskaya',
    'kurskaia-oblast': 'Kurskaya',
    'leningradskaia-oblast': 'Leningradskaya',
    'lipetskaia-oblast': 'Lipetskaya',
    'magadanskaia-oblast': 'Magadanskaya',
    'moskva': 'Moskva',
    'moskovskaia-oblast': 'Moskovskaya',
    'murmanskaia-oblast': 'Murmanskaya',
    'nenetskii-avtonomnyi-okrug': 'Nenetskiy',
    'nizhegorodskaia-oblast': 'Nizhegorodskaya',
    'novgorodskaia-oblast': 'Novgorodskaya',
    'novosibirskaia-oblast': 'Novosibirskaya',
    'omskaia-oblast': 'Omskaya',
    'orenburgskaia-oblast': 'Orenburgskaya',
    'orlovskaia-oblast': 'Orlovskaya',
    'penzenskaia-oblast': 'Penzenskaya',
    'permskii-krai': 'Primorskiy_kray',
    'primorskii-krai': 'Primorskiy_kray',
    'pskovskaia-oblast': 'Pskovskaya',
    'respublika-adygeia-adygeia': 'Adygeya',
    'respublika-altai': 'Altay',
    'respublika-bashkortostan': 'Bashkortostan',
    'respublika-buriatiia': 'Buryatiya',
    'respublika-dagestan': 'Dagestan',
    'respublika-ingushetiia': 'Ingushskaya',
    'respublika-kalmykiia': 'Kalmykiya',
    'respublika-kareliia': 'Kareliya',
    'respublika-komi': 'Komi',
    'respublika-marii-el': 'Mariy_El',
    'respublika-mordoviia': 'Mordoviya',
    'respublika-sakha-iakutiia': 'Sakha_Yakutiya',
    'respublika-severnaia-osetiia-alaniia': 'Severnaya_Osetiya_Alaniya',
    'respublika-tatarstan-tatarstan': 'Tatarstan',
    'respublika-tyva': 'Tyva',
    'respublika-khakasiia': 'Khakasiya',
    'rostovskaia-oblast': 'Rostovskaya',
    'riazanskaia-oblast': 'Ryazanskaya',
    'samarskaia-oblast': 'Samarskaya',
    'sankt-peterburg': 'Sankt-Peterburg',
    'saratovskaia-oblast': 'Saratovskaya',
    'sakhalinskaia-oblast': 'Sakhalinskaya',
    'sverdlovskaia-oblast': 'Sverdlovskaya',
    'smolenskaia-oblast': 'Smolenskaya',
    'stavropolskii-krai': 'Stavropol-skiy_kray',
    'tambovskaia-oblast': 'Tambovskaya',
    'tverskaia-oblast': 'Tverskaya',
    'tomskaia-oblast': 'Tomskaya',
    'tulskaia-oblast': 'Tul-skaya',
    'tiumenskaia-oblast': 'Tyumenskaya',
    'udmurtskaia-respublika': 'Udmurtskaya',
    'ulianovskaia-oblast': 'Ul-yanovskaya',
    'khabarovskii-krai': 'Khabarovskiy-kray',
    'khanty-mansiiskii-avtonomnyi-okrug-iugra': 'Khanty-Mansiyskiy',
    'cheliabinskaia-oblast': 'Chelyabinskaya',
    'chechenskaia-respublika': 'Chechenskaya',
    'chuvashskaia-respublika-chuvashiia': 'Chuvashskaya',
    'chukotskii-avtonomnyi-okrug': 'Chukotskiy',
    'iamalo-nenetskii-avtonomnyi-okrug': 'Yamalo-Nenetskiy',
    'iaroslavskaia-oblast': 'Yaroslavskaya'
}

### Download every file and store it `./data/*.geojson` file
NOTE: `geojson_files` will be changed in next cell

In [6]:
import requests
from pathlib import Path

geojson_files = []
Path.mkdir(Path('data'), exist_ok=True)

for file_link in tqdm(download_links):
    file_data = requests.get(file_link).content
    with open(file_link.replace('https://cms.dtp-stat.ru/media/opendata/', 'data/'), 'wb+') as file:
        geojson_files.append(file.name)
        file.write(file_data)


  0%|          | 0/85 [00:00<?, ?it/s]

## Loading data from file and convert it to `CarAccident` class

### Load data from file and store it `geojson_objects` list

In [136]:
import json
import geojson
from shapely.geometry import shape, Point
import chardet

# for those who already have the data downloaded, so you can skip the previous cell
geojson_files = [str(file) for file in Path.cwd().glob('data/*.geojson')]
Path.mkdir(Path('data/fixed'), exist_ok=True)
geojson_objects = []

for file in tqdm(geojson_files):
    with open(file, 'r', encoding='utf-8') as f:
            try:
                region_name = eq_regions[Path(file).name.replace('.geojson', '')]
            except KeyError:
                continue
            # TODO: change all shape-files encoding to utf-8 and remove chardet
            with open(f'Russia_geoJSON/{region_name}.geojson', 
                      'r', 
                      encoding=chardet.detect(open(f'Russia_geoJSON/{region_name}.geojson', 'rb').read(1500))['encoding']) as poly_file:
                shape_data = shape(json.loads(poly_file.read())['geometry'])
                data = json.loads(f.read())
                _data = list(data['features'])
                for feature in tqdm(data['features']):
                    try:
                        if not shape_data.contains(Point(feature['geometry']['coordinates'])):
                            _data.remove(feature)
                    except Exception as e:
                        _data.remove(feature)
                with open(f'data/fixed/{region_name}.geojson', 'w+') as dump_data:
                    data['features'] = _data
                    data_to_dump = json.dumps(data, ensure_aexitscii=False).encode('utf8')
                    dump_data.write(data_to_dump.decode())
            geojson_objects.append(geojson.FeatureCollection(data['features']))


  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/28643 [00:00<?, ?it/s]

  0%|          | 0/8444 [00:00<?, ?it/s]

  0%|          | 0/20880 [00:00<?, ?it/s]

  0%|          | 0/30759 [00:00<?, ?it/s]

...

  0%|          | 0/14292 [00:00<?, ?it/s]

  0%|          | 0/14143 [00:00<?, ?it/s]

  0%|          | 0/6770 [00:00<?, ?it/s]

  0%|          | 0/24133 [00:00<?, ?it/s]

  0%|          | 0/22301 [00:00<?, ?it/s]

In [112]:
for i in data['features']:
    print(data['features'][0])
    data['features'].remove(i)
    print(data['features'][0])
    break
print(data['features'][0])

{
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [39.6889, 47.2081]},
    'properties': {
        'id': 2433824,
        'tags': ['Дорожно-транспортные происшествия'],
        'light': 'Светлое время суток',
        'point': {'lat': 47.2081, 'long': 39.6889},
        'nearby': ['Многоквартирные жилые дома', 'Административные здания'],
        'region': 'Ростов-на-Дону',
        'scheme': '220',
        'address': 'г Ростов-на-Дону, ул Портовая, 76',
        'weather': ['Дождь'],
        'category': 'Столкновение',
        'datetime': '2019-05-25 15:20:00',
        'severity': 'Легкий',
        'vehicles': [
            {
                'year': 2012,
                'brand': 'DAEWOO',
                'color': 'Белый',
                'model': 'Nexia',
                'category': 'В-класс (малый) до 3,9 м',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': ['Несоответствие скорости конкретным условиям движения'],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 4
                    }
                ]
            },
            {
                'year': 2008,
                'brand': 'HYUNDAI',
                'color': 'Серый',
                'model': 'Accent',
                'category': 'В-класс (малый) до 3,9 м',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': ['Несоблюдение требований ОСАГО'],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 25
                    },
                    {
                        'role': 'Пассажир',
                        'gender': 'Женский',
                        'violations': [],
                        'health_status': 'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо 
которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его 
фактического прохождения)',
                        'years_of_driving_experience': None
                    },
                    {
                        'role': 'Пассажир',
                        'gender': 'Мужской',
                        'violations': [],
                        'health_status': 'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо 
которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его 
фактического прохождения)',
                        'years_of_driving_experience': None
                    }
                ]
            }
        ],
        'dead_count': 0,
        'participants': [],
        'injured_count': 2,
        'parent_region': 'Ростовская область',
        'road_conditions': ['Мокрое'],
        'participants_count': 4,
        'participant_categories': ['Все участники']
    }
}

{
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [39.7442, 47.2867]},
    'properties': {
        'id': 2442272,
        'tags': ['Дорожно-транспортные происшествия'],
        'light': 'В темное время суток, освещение включено',
        'point': {'lat': 47.2867, 'long': 39.7442},
        'nearby': ['Многоквартирные жилые дома'],
        'region': 'Ростов-на-Дону',
        'scheme': '910',
        'address': 'г Ростов-на-Дону, ул Штахановского, 12',
        'weather': ['Пасмурно'],
        'category': 'Наезд на стоящее ТС',
        'datetime': '2018-05-25 20:35:00',
        'severity': 'Легкий',
        'vehicles': [
            {
                'year': 1982,
                'brand': 'VOLVO',
                'color': 'Белый',
                'model': 'Прочие модели Volvo',
                'category': 'Прочие одноярусные',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': [],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 16
                    }
                ]
            },
            {
                'year': 2005,
                'brand': 'Прочие марки мотоциклов',
                'color': 'Белый',
                'model': 'Прочие марки мотоциклов',
                'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': ['Неправильный выбор дистанции'],
                        'health_status': 'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо 
которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его 
фактического прохождения)',
                        'years_of_driving_experience': 6
                    }
                ]
            },
            {
                'year': 2017,
                'brand': 'RENAULT',
                'color': 'Белый',
                'model': 'Logan',
                'category': 'В-класс (малый) до 3,9 м',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': [],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 4
                    }
                ]
            }
        ],
        'dead_count': 0,
        'participants': [],
        'injured_count': 1,
        'parent_region': 'Ростовская область',
        'road_conditions': [
            'Неисправное освещение',
            'Сухое',
            'Отсутствие, плохая различимость горизонтальной разметки проезжей части'
        ],
        'participants_count': 3,
        'participant_categories': ['Мотоциклисты', 'Все участники']
    }
}

{
    'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [39.7442, 47.2867]},
    'properties': {
        'id': 2442272,
        'tags': ['Дорожно-транспортные происшествия'],
        'light': 'В темное время суток, освещение включено',
        'point': {'lat': 47.2867, 'long': 39.7442},
        'nearby': ['Многоквартирные жилые дома'],
        'region': 'Ростов-на-Дону',
        'scheme': '910',
        'address': 'г Ростов-на-Дону, ул Штахановского, 12',
        'weather': ['Пасмурно'],
        'category': 'Наезд на стоящее ТС',
        'datetime': '2018-05-25 20:35:00',
        'severity': 'Легкий',
        'vehicles': [
            {
                'year': 1982,
                'brand': 'VOLVO',
                'color': 'Белый',
                'model': 'Прочие модели Volvo',
                'category': 'Прочие одноярусные',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': [],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 16
                    }
                ]
            },
            {
                'year': 2005,
                'brand': 'Прочие марки мотоциклов',
                'color': 'Белый',
                'model': 'Прочие марки мотоциклов',
                'category': 'Мопеды с двигателем внутреннего сгорания менее 50 см. куб.',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': ['Неправильный выбор дистанции'],
                        'health_status': 'Раненый, находящийся (находившийся)  на амбулаторном лечении, либо 
которому по характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его 
фактического прохождения)',
                        'years_of_driving_experience': 6
                    }
                ]
            },
            {
                'year': 2017,
                'brand': 'RENAULT',
                'color': 'Белый',
                'model': 'Logan',
                'category': 'В-класс (малый) до 3,9 м',
                'participants': [
                    {
                        'role': 'Водитель',
                        'gender': 'Мужской',
                        'violations': [],
                        'health_status': 'Не пострадал',
                        'years_of_driving_experience': 4
                    }
                ]
            }
        ],
        'dead_count': 0,
        'participants': [],
        'injured_count': 1,
        'parent_region': 'Ростовская область',
        'road_conditions': [
            'Неисправное освещение',
            'Сухое',
            'Отсутствие, плохая различимость горизонтальной разметки проезжей части'
        ],
        'participants_count': 3,
        'participant_categories': ['Мотоциклисты', 'Все участники']
    }
}

### Create a dataclasses for better data analysis
NOTE that dataclasses are inherits from `pydantic.BaseModel`

In [8]:
from pydantic import BaseModel

class Participant(BaseModel):
    role: str | None
    gender: str | None
    violations: list | None
    health_status: str | None
    years_of_driving_experience: int | None

class Vehicle(BaseModel):
    year: int | None
    brand: str | None
    color: str | None
    category: str | None
    participants: list[Participant] | None

class CarAccident(BaseModel):
    id: int | None
    tags: list | None
    light: str | None
    point: dict | None
    nearby: list | None
    region: str | None
    scheme: str | None
    address: str | None
    weather: list | None 
    category: str | None
    datetime: str | None
    severity: str | None
    vehicles: list[Vehicle] | None 
    dead_count: int | None
    participants: list[Participant] | None
    injured_count: int | None
    parent_region: str | None 
    road_conditions: list | None
    participants_count: int | None
    participant_categories: list | None

In [9]:
# load properties from geojson_objects to CarAccident objects
car_accidents = []
for geojson_object in tqdm(geojson_objects):
    for feature in geojson_object['features']:
        car_accident = CarAccident(**feature['properties'])
        car_accidents.append(car_accident)

print(car_accidents[0])

  0%|          | 0/85 [00:00<?, ?it/s]

CarAccident(
    id=2431792,
    tags=['Дорожно-транспортные происшествия'],
    light='Светлое время суток',
    point={'lat': 47.2697, 'long': 39.7142},
    nearby=['Жилые дома индивидуальной застройки', 'Регулируемый перекресток'],
    region='Ростов-на-Дону',
    scheme='500',
    address='г Ростов-на-Дону, ул Фурмановская, 76 56',
    weather=['Ясно'],
    category='Столкновение',
    datetime='2018-02-21 09:00:00',
    severity='Легкий',
    vehicles=[
        Vehicle(
            year=2012,
            brand='ВАЗ',
            color='Серый',
            category='В-класс (малый) до 3,9 м',
            participants=[
                Participant(
                    role='Водитель',
                    gender='Мужской',
                    violations=['Несоблюдение очередности проезда перекрестков'],
                    health_status='Не пострадал',
                    years_of_driving_experience=5
                )
            ]
        ),
        Vehicle(
            year=2012,
            brand='RENAULT',
            color='Белый',
            category='В-класс (малый) до 3,9 м',
            participants=[
                Participant(
                    role='Водитель',
                    gender='Мужской',
                    violations=[],
                    health_status='Раненый, находящийся (находившийся)  на амбулаторном лечении, либо которому по 
характеру полученных травм обозначена необходимость амбулаторного лечения (вне зависимости от его фактического 
прохождения)',
                    years_of_driving_experience=2
                )
            ]
        )
    ],
    dead_count=0,
    participants=[],
    injured_count=1,
    parent_region='Ростовская область',
    road_conditions=['Сухое'],
    participants_count=2,
    participant_categories=['Все участники']
)

In [10]:
# create a set variable for every list of CarAccident objects
tags_set = set()
light_set = set()
nearby_set = set()
region_set = set()
scheme_set = set()
address_set = set()
weather_set = set()
category_set = set()
severity_set = set()
dead_count_set = set()
injured_count_set = set()
parent_region_set = set()
road_conditions_set = set()
participants_count_set = set()
participant_categories_set = set()

participant_role_set = set()
participant_gender_set = set()
participant_violations_set = set()
participant_health_status_set = set()

vehicle_year_set = set()
vehicle_brand_set = set()
vehicle_color_set = set()  # xD
vehicle_category_set = set()

for accident in tqdm(car_accidents):
    tags_set.update(accident.tags)
    light_set.update([accident.light])
    nearby_set.update(accident.nearby)
    region_set.update([accident.region])
    scheme_set.update([accident.scheme])
    address_set.update([accident.address])
    weather_set.update(accident.weather)
    category_set.update([accident.category])
    severity_set.update([accident.severity])
    dead_count_set.update([accident.dead_count])
    injured_count_set.update([accident.injured_count])
    parent_region_set.update([accident.parent_region])
    road_conditions_set.update(accident.road_conditions)
    participants_count_set.update([accident.participants_count])
    participant_categories_set.update(accident.participant_categories)
    [participant_violations_set.update(participant.violations) for vehicle in accident.vehicles for participant in vehicle.participants]
    participant_role_set.update(participant.role for vehicle in accident.vehicles for participant in vehicle.participants)
    participant_gender_set.update(participant.gender for vehicle in accident.vehicles for participant in vehicle.participants)
    participant_health_status_set.update(participant.health_status for vehicle in accident.vehicles for participant in vehicle.participants)
    vehicle_year_set.update(str(vehicle.year) for vehicle in accident.vehicles)
    vehicle_brand_set.update(vehicle.brand for vehicle in accident.vehicles)
    vehicle_color_set.update(vehicle.color for vehicle in accident.vehicles)
    vehicle_category_set.update(vehicle.category for vehicle in accident.vehicles)

  0%|          | 0/1254065 [00:00<?, ?it/s]

In [15]:
# print(f'Tags: {tags_set}')
# print(f'Weather: {weather_set}')
# print(f'Participant categories: {participant_categories_set}')
# print(f'Road conditions: {road_conditions_set}')
# print(f'Participant roles: {participant_violations_set}')
# print(f'Vehicle colors: {vehicle_color_set}')

with open('sets.txt', 'w+', encoding='utf-8') as f:
    f.write(f'tags_set = {tags_set}\n\n')
    f.write(f'tags_set = {tags_set}\n\n')
    f.write(f'light_set = {light_set}\n\n')
    f.write(f'nearby_set = {nearby_set}\n\n')
    f.write(f'scheme_set = {scheme_set}\n\n')
    f.write(f'weather_set = {weather_set}\n\n')
    f.write(f'category_set = {category_set}\n\n')
    f.write(f'severity_set = {severity_set}\n\n')
    f.write(f'dead_count_set = {dead_count_set}\n\n')
    f.write(f'injured_count_set = {injured_count_set}\n\n')
    f.write(f'road_conditions_set = {road_conditions_set}\n\n')
    f.write(f'participants_count_set = {participants_count_set}\n\n')
    f.write(f'participant_categories_set = {participant_categories_set}\n\n')
    f.write(f'participant_role_set = {participant_role_set}\n\n')
    f.write(f'participant_gender_set = {participant_gender_set}\n\n')
    f.write(f'participant_violations_set = {participant_violations_set}\n\n')
    f.write(f'participant_health_status_set = {participant_health_status_set}\n\n')
    f.write(f'vehicle_year_set = {vehicle_year_set}\n\n')
    f.write(f'vehicle_brand_set = {vehicle_brand_set}\n\n')
    f.write(f'vehicle_color_set = {vehicle_color_set}\n\n')
    f.write(f'vehicle_category_set = {vehicle_category_set}\n\n')